# Week 1: design and data representation
This week's tutorial/assignment will be about the basics of pattern analysis. More specifically, we'll discuss how the different types of pattern analyses in terms of different *designs* (principally within- and between-subject designs) and how to transform and represent fMRI data optimally for pattern analyses. Additionally, we'll discuss some new programming concepts (mostly object-oriented programming), as this will become more and more important as you're going to learn how to program everything yourself in Python. 

Before



## Learning objectives

## Contents
- Designs for pattern analyses
- Introduction to object-oriented programming
- Data representation

## 1. Designs for pattern analyses
There are many ways in which you can categorize different types of pattern analyses (a topic which you'll work on in Thursdays's seminar), but one of the most basic categorizations is in terms of whether analyses are **within-subject** or **between-subject**. The major distinction revolves around whether you want to investigate an (experimental) variable that varies or is manipulated within subjects or that varies across subjects (i.e. individual differences or experimental between-subject designs). 

Theoretically, both machine learning (ML) and RSA-type analyses can operate both on a within-subject and between-subject level. However, in practice, most pattern analyses (both ML and RSA) are within-subject, partly because between-subject analyses require much more data (this will be discussed in more detail in the next weeks). In what follows, we'll describe these two types (within/between) of analyses in more detail and discuss how the corresponding designs look like.

### 1.1. Within-subject analyses
Within-subject analyses aim to investigate the representation of factors/variables that vary within an experiment. This is, of course, still a very broad definition. To clarify, take for example the following working-memory experiment (see image below for a schematic representation):

Subjects are shown a configuration of bars on the screen in different orientations. After this stimulus, subjects are shown a blank screen with a fixation cross ("retention interval") in which they have to memory the configuration of bars. After 4 seconds, they are shown the configuration of bars again, but here's the catch: the configuration can be exactly the same as the first time or *one* bar can be in a different orientation - let's call this factor **same/different**. Then, a black screen will follow after which subjects are prompted to respond whether it's the same configuration as before (respond left) or different (repond right). Let's call this factor **left/right**. The subject can, of course, be correct or incorrect in their response - let's call this factor **correct/incorrect**. Lastly, in fact, the configuration can consist of 3 bars or of 6 bars in total, which of course impacts the "memory load" of the task - let's call this factor **low/high**. 

<img src="within_subject_example.png">

As you can see, in this relatively simple experiment, there are already several factors that we can investigate. To summarize, we can investigate the relation between brain patterns and:
- representation of whether a stimulus has changed or not (factor: same/different);
- representation of left/right button presses (factor: left/right);
- representation of correct/incorrect responses (factor: correct/incorrect);
- representation of low/high working memory load (factor: low/high);


<div class='alert alert-info'>
**ToThink**: A

</div>

### 1.2. Between-subject analyses